In [1]:
from openai import OpenAI

In [2]:
client = OpenAI()

In [3]:
# STEP 1: Creating the assistant

In [5]:
assistant = client.beta.assistants.create(name='Mortgage Bot',
                                          instructions='You use Python code to help answer questions about mortgage and interest payments.',
                                          tools = [{'type':'code_interpreter'}],
                                          model='gpt-3.5-turbo'
                                         )

                                          

In [6]:
#STEP 2: Thread Creation

In [7]:
thread = client.beta.threads.create()

In [8]:
#STEP 3: Add message to thread

In [9]:
message = client.beta.threads.messages.create(thread_id=thread.id,
                                              role='user',
                                              content='I want to buy a house that costs £2.1 million on a 30 year fixed loan at 7.8% interest. What will my monthly payments be?'
                                             )

In [10]:
#STEP 4: Run thread w/ Assistant

In [15]:
run = client.beta.threads.runs.create(thread_id=thread.id,
                               assistant_id=assistant.id,
                               instructions='Give a detailed analysis and considerations regarding the mortgage payments.'
                              )
                        

In [16]:
import time

while run.status != 'completed':
    print(run.status)
    run = client.beta.threads.runs.retrieve(thread_id = thread.id, run_id=run.id)
    time.sleep(5)

queued
in_progress


In [17]:
#STEP 5: Display response

In [27]:
messages = client.beta.threads.messages.list(thread_id=thread.id,
                                            order='asc',after=message.id)

In [28]:
# messages

In [29]:
def display_thread_messages(messages):
    for thread_message in messages.data:
        print(thread_message.content[0].text.value)
        print('\n')

In [30]:
display_thread_messages(messages)

To calculate the monthly mortgage payments for a fixed-rate loan, we can use the formula for the monthly payment of a fixed-rate mortgage:

\[ M = P \times \frac{r(1+r)^n}{(1+r)^n-1} \]

Where:
- \(M\) = monthly mortgage payment
- \(P\) = principal loan amount (loan amount)
- \(r\) = monthly interest rate (annual interest rate divided by 12 months)
- \(n\) = total number of payments (loan term in years multiplied by 12 months)

We can plug in the given values and calculate the monthly mortgage payments. Let's proceed with the calculation.


The monthly mortgage payment for a £2.1 million house on a 30-year fixed loan at 7.8% interest rate would be approximately £15,117.28.


This monthly payment includes both the principal and interest components of the mortgage. Over the 30-year term, you would make a total of 360 monthly payments. 

It's important to note that this calculation does not include other potential costs associated with homeownership, such as property taxes, homeowners ins

In [34]:
def wait_on_run(run,thread):
    while run.status == 'queued' or run.status == 'in_progress':
        run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
        time.sleep(0.5)
        print(run.status)
    return run

In [36]:
# client.beta.threads.messages.list(thread_id=thread.id)

In [37]:
message = client.beta.threads.messages.create(thread_id=thread.id,
                                                role='user',
                                                  content='What is i put a down payments of £200k on the house, how would this change my monthly payments?')

In [38]:
#EXECUTE RUN --> ASSISTANT +THREAD
run = client.beta.threads.runs.create(thread_id=thread.id,assistant_id=assistant.id)

#WAIT UNTIL RUN IS COMPLETED
wait_on_run(run,thread)

messages = client.beta.threads.messages.list(thread_id=thread.id,
                                             order='asc',
                                             after=message.id)

display_thread_messages(messages)

in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
completed
If you make a down payment on the house, the principal loan amount would be reduced by the amount of the down payment. The new principal loan amount would be £2,100,000 - £200,000 = £1,900,000.

We can now recalculate the monthly mortgage payment using the reduced principal loan amount. Let's calculate the new monthly payment.


With a down payment of £200,000 on the house, the new monthly mortgage payment would be approximately £13,677.54. 

This down payment has reduced the total amount borrowed, which has lowered the monthly payment amount.




In [39]:
run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id,run_id=run.id)

In [40]:
# for step in run_steps:
   # print(step.step_details)

MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_lMcsssj9OQBCW53KLBwweRAM'), type='message_creation')
ToolCallsStepDetails(tool_calls=[CodeInterpreterToolCall(id='call_Fc60Jj8idwXymT9clgyZMc9b', code_interpreter=CodeInterpreter(input='# New principal loan amount after down payment\nnew_principal = principal - 200000\n\n# Calculating monthly mortgage payment with the down payment\nnew_monthly_payment = new_principal * (monthly_interest_rate * (1 + monthly_interest_rate) ** total_payments) / ((1 + monthly_interest_rate) ** total_payments - 1)\n\nnew_monthly_payment', outputs=[]), type='code_interpreter')], type='tool_calls')
MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_ZqF2Mq3GYjCDzraANShv7lzk'), type='message_creation')


In [41]:
response = client.beta.assistants.delete(assistant.id)

In [42]:
response

AssistantDeleted(id='asst_J8tybTeH5IKZ86bQvtVp1RBn', deleted=True, object='assistant.deleted')